In [1]:
import requests
import pandas as pd
import time

# Initialize an empty DataFrame to store the data
data = pd.DataFrame()

# Loop to fetch 100000 rows of data in increments of 1000
for offset in range(0, 1000, 1000):
    url = f"https://data.cityofchicago.org/resource/4ijn-s7e5.csv?$limit=1000&$offset={offset}"
    success = False
    retries = 3
    
    while not success and retries > 0:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Check if the request was successful
            temp_data = pd.read_csv(url)
            data = pd.concat([data, temp_data], ignore_index=True)
            success = True
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data at offset {offset}: {e}")
            retries -= 1
            time.sleep(5)  # Wait for 5 seconds before retrying
    
    if not success:
        print(f"Failed to fetch data at offset {offset} after 3 retries.")

# Print the number of rows fetched
print(f"Total rows fetched: {len(data)}")



Total rows fetched: 1000


In [2]:
data.sample(1)


,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
228,2615892,GOMEZ TACOS REST,GOMEZ TACOS REST,17434,Restaurant,Risk 1 (High),3016 E 91ST ST,CHICAGO,IL,60617.0,2025-04-16T00:00:00.000,Canvass,Pass w/ Conditions,16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...,41.730197,-87.550474,"\n, \n(41.73019682889687, -87.55047439684799)"


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   inspection_id    1000 non-null   int64  
 1   dba_name         1000 non-null   object 
 2   aka_name         995 non-null    object 
 3   license_         1000 non-null   int64  
 4   facility_type    980 non-null    object 
 5   risk             999 non-null    object 
 6   address          1000 non-null   object 
 7   city             999 non-null    object 
 8   state            1000 non-null   object 
 9   zip              999 non-null    float64
 10  inspection_date  1000 non-null   object 
 11  inspection_type  1000 non-null   object 
 12  results          1000 non-null   object 
 13  violations       676 non-null    object 
 14  latitude         994 non-null    float64
 15  longitude        994 non-null    float64
 16  location         994 non-null    object 
dtypes: float64(3), 

In [4]:
print(data['violations'].iloc[0])
print(data['violations'].iloc[20])

print(data['violations'].iloc[123])

nan
55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: 6-501.11 OBSERVED STAINED CEILING TILES IN REAR DISH/ DINING AREA OF FACILITY. INSTRUCTED MANAGER TO REPLACE STAINED CEILING TILES.
1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES - Comments: OBSERVED PIC WITHOUT A VALID CITY OF CHICAGO SANITATION CERTIFICATE AS REQUIRED. | 2. CITY OF CHICAGO FOOD SERVICE SANITATION CERTIFICATE - Comments: OBSERVED NO VALID CITY OF CHICAGO CERTIFIED FOODSERVICE MANAGER ON DUTY AT THIS TIME WHILE TCS FOODS SUCH AS CHICKEN BEING PREPARED, HANDLED AND SERVED. INSTRUCED MANAGER TO PROVIDE AND MAINTAIN AT ALL TIMES. PRIORITY FOUNDATION CITATION ISSUED #7-38-012. | 33. PROPER COOLING METHODS USED; ADEQUATE EQUIPMENT FOR TEMPERATURE CONTROL - Comments: OBSERVED THE PREP COOLER BEHIND THE SERVICE COUNTER WITH AN IMPROPER INTERIOR AMBIENT TEMP OF 56.6F SAID COOLER HOLDS CONDIMENTS. COOLER TAGGED / HELD FOR INSPECTION PRIORITY CITATION ISSUED #7-38-005. | 51. PLUMBING 

In [ ]:
import pandas as pd
import re

# function to clean

def extract_violation_numbers(violations):
    if pd.isna(violations) or violations == "nan":
        return []
    return [str(int(num)) for num in re.findall(r'\b\d+\b', violations)] 

# violations
data['violations'] = data['violations'].apply(lambda x: ' '.join(extract_violation_numbers(str(x))))


In [6]:
print(data['violations'].iloc[0])

print(data['violations'].iloc[2])

print(data['violations'].iloc[324])

print(data['violations'].iloc[4])



55
39 6 55
2 7 38 12 8 9 2023 10 7 38 30 16 7 38 25 48 7 38 25 51


In [ ]:
# function to clean

def extract_violation_numbers(violations):
    if pd.isna(violations) or violations == "nan":
        return []
    return [str(int(num)) for num in re.findall(r'\b\d+\b', violations)] 



def clean(df):
    df['inspection_date'] = pd.to_datetime(df['inspection_date']).dt.strftime('%Y-%m-%d')

    # fix city
    df['city'] = df['city'].str.replace(r'(?i)chicago', 'CHICAGO', regex=True)

    # fix facility type
    df['facility_type'] = df['facility_type'].str.replace(r'(?i)(.*years.*|.*school.*|.*daycare.*|.*youth.*|.*shcool.*|.*charter.*|.*child.*|.*children.*|.*student.*)', 'Child or Student Facilities', regex=True)
    df['facility_type'] = df['facility_type'].str.replace(r'(?i)(.*store.*|.*pharmacy.*|.*shop.*|.*grocery.*|.*liquor.*|.*wholesale.*|.*mart.*|.*retail.*|.*service.*|.*gas.*)', 'Retail', regex=True)
    df['facility_type'] = df['facility_type'].str.replace(r'(?i)(.*restaurant.*|.*catering.*|.*diner.*|.*shared.*|.*kitchen.*|.*pantry.*|.*bakery.*|.*tavern.*|.*coffee.*|.*ice.*|.*deli.*|.*sushi.*|.*tea.*|.*bakery.*|.*bar.*|.*hookah.*)', 'Food Service', regex=True)
    df['facility_type'] = df['facility_type'].str.replace(r'(?i)(.*mobile.*)', 'Mobile', regex=True)
    df['facility_type'] = df['facility_type'].str.replace(r'(?i)(.*event.*|.*venue.*|.*banquet.*|.*church.*)', 'Event or Venue', regex=True)
    df['facility_type'] = df['facility_type'].str.replace(r'(?i)(.*nursing.*|.*elder.*|.*assisted.*|.*senior.*|.*care.*)', 'Elderly Living', regex=True)
    df['facility_type'] = df['facility_type'].where(
        df['facility_type'].isin(['Child or Student Facilities', 'Retail', 'Food Service', 'Mobile', 'Event or Venue', 'Elderly Living']),
        'Other'
    )

    # violations
    data['violations'] = data['violations'].apply(lambda x: ' '.join(extract_violation_numbers(str(x))))


    df = df.drop(columns=['dba_name', 'license_', 'city', 'state', 'location'])
    df = df[df['results'] != 'No Entry']
    df = df.dropna()
    return df


In [8]:
# test
data = clean(data)


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 932 entries, 0 to 999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   inspection_id    932 non-null    int64  
 1   aka_name         932 non-null    object 
 2   facility_type    932 non-null    object 
 3   risk             932 non-null    object 
 4   address          932 non-null    object 
 5   zip              932 non-null    float64
 6   inspection_date  932 non-null    object 
 7   inspection_type  932 non-null    object 
 8   results          932 non-null    object 
 9   violations       932 non-null    object 
 10  latitude         932 non-null    float64
 11  longitude        932 non-null    float64
dtypes: float64(3), int64(1), object(8)
memory usage: 94.7+ KB


In [10]:
print("Unique types:", data['facility_type'].unique())

Unique types: ['Food Service' 'Mobile' 'Child or Student Facilities' 'Retail' 'Other'
 'Elderly Living']


In [11]:
data.sample(5)

,inspection_id,aka_name,facility_type,risk,address,zip,inspection_date,inspection_type,results,violations,latitude,longitude
163,2615964,THE CHICAGO RIB HOUSE,Food Service,Risk 1 (High),3851 S MICHIGAN AVE,60653.0,2025-04-17,Short Form Complaint,Pass,38 51 55,41.824043,-87.622947
528,2615341,MiMi's Tacos And French Fries,Food Service,Risk 1 (High),1352 W TAYLOR ST,60607.0,2025-04-08,Canvass Re-Inspection,Pass,,41.869454,-87.661338
197,2615801,Vanderpoel Magnet,Child or Student Facilities,Risk 1 (High),9510 S Prospect (1700W),60643.0,2025-04-16,Canvass Re-Inspection,Pass,,41.720782,-87.665585
31,2616165,DIPPIN' DOTS ICE CREAM,Mobile,Risk 2 (Medium),324 N Leavitt ST,60612.0,2025-04-22,License,Pass,,41.887434,-87.681849
706,2615058,WEI DAO CHENG DOU,Food Service,Risk 1 (High),2105 S JEFFERSON ST,60616.0,2025-04-03,License Re-Inspection,Pass,10 6 301 14 36 4 204 112 39 3 307 11 48 4 301 ...,41.854071,-87.641600
